In [ ]:
#%env JOBLIB_TEMP_FOLDER=/tmp

In [ ]:
import gc
import numba
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import stats
from tqdm import tqdm
from joblib import Parallel,delayed
import time
import re
from collections import OrderedDict
import category_encoders as ce

import sys
sys.path.append("../utils")
from memory import reduce_mem_usage

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=16, progress_bar=True, verbose=2, use_memory_fs=False)

In [ ]:
OUTPUT_PATH = Path("../data/processed/dsv05")

if not OUTPUT_PATH.exists():
    OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

this datasets aggreates the features over the time dimension

- takes as base this dataset: https://www.kaggle.com/datasets/raddar/amex-data-integer-dtypes-parquet-format
- feat engineering from here: https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created
- lag features idea from here: https://www.kaggle.com/code/thedevastator/lag-features-are-all-you-need/

In [ ]:
@numba.njit()
def compute_slope(x, y):
    x_mean = x.mean()
    y_mean = y.mean()
    return np.sum((x-x_mean)*(y-y_mean)) / np.sum((x-x_mean)**2)

def compute_slope_cols(df, customer_ID, num_features):
    n = len(df)
    if n > 2:
        x = np.arange(n)
        _df = df[num_features].fillna(method="ffill", axis=0).fillna(method="bfill", axis=0)
        r = _df[num_features].apply(lambda y: compute_slope(x, y.values))
        r = r.to_dict()
    else:
        r = df[num_features].apply(lambda y: 0)
        r = r.to_dict()
    r["customer_ID"] = customer_ID
    return r

def mode_1st(x):
    return x.value_counts().index[0]

def mode_2nd(x):
    try: return x.value_counts().index[1]
    except: return -1 

numba.njit()
def compute_last_diff(array):
    if len(array) <= 1:
        return np.nan
    else:
        return array[-1]-array[-2]
    
def compute_last_diff_series(df, col):
    r = df.groupby("customer_ID")[col].apply(lambda x: compute_last_diff(x.values))
    r.name = f"{r.name}_diff"
    return r

In [ ]:
def min_max_position(df, customer_ID, num_features):
    out = OrderedDict()
    out["customer_ID"] = customer_ID
    for col in num_features:
        idxmin = np.argmin(df[col].values[::-1])
        idxmax = np.argmax(df[col].values[::-1])
        out[f"{col}_idxmin"] = idxmin
        out[f"{col}_idxmax"] = idxmax
    return out

In [ ]:
def encode_categoricals(dataframe, encoder=None):
    categoricals = [
        'B_30', 'B_38', 'D_63', 'D_64', 'D_66', 'D_68', 
        'D_114', 'D_116', 'D_117', 'D_120', 'D_126',
    ]
    
    if encoder is None:
        print("[INFO] fitting the encoder")
        encoder = ce.one_hot.OneHotEncoder(cols=categoricals)
        encoder.fit(dataframe[categoricals])
        
    out = encoder.transform(dataframe[categoricals]).astype(np.int8)
    ohe_cols = encoder.get_feature_names()
    
    dataframe.drop(categoricals, axis=1, inplace=True)
    dataframe = pd.concat([dataframe, out], axis=1)

    gc.collect()
    
    return dataframe, encoder, ohe_cols

#@numba.njit()
def compute_last_observed(series):
    idx = np.nonzero(series.values[::-1])[0]
    if len(idx)==0:
        return 100
    else:
        return idx[0]

In [ ]:
def count_consecutives(df, customer_ID, num_features):
    out = OrderedDict()
    out["customer_ID"] = customer_ID
    for col in num_features:
        out[col] = np.sum(np.cumprod(df[col].values[::-1]))
    return out

In [ ]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

def compute_antiquity(df):
    return (df.S_2.dt.year.values[-1] - df.S_2.dt.year.values[0])*12 + (df.S_2.dt.month.values[-1] - df.S_2.dt.month.values[0])

In [ ]:
def first_nan_position(series):
    idx = np.nonzero(series.values)
    if len(idx)==0:
        return 100
    else:
        return idx[0]
    
def last_nan_position(series):
    idx = np.nonzero(series.values[::-1])
    if len(idx)==0:
        return 100
    else:
        return idx[0]

In [ ]:
# references: 
# https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created
# https://www.kaggle.com/code/cdeotte/xgboost-starter-0-793
# after pay feats: https://www.kaggle.com/code/jiweiliu/rapids-cudf-feature-engineering-xgb
# other lag features: https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977

def remove_noise(df): 
    # removes noise from float columns
    float_cols = df.dtypes[df.dtypes == "float32"].index
    print(f"[INFO] number of float cols to reduce noise: {len(float_cols)}")
    for col in float_cols:
        df[col] = df[col].round(decimals=2)
    return df

def build_features(df, ohe_cols=None):
    
    df = df.copy()
    df["S_2"] = pd.to_datetime(df.S_2)

    n_customers = df["customer_ID"].nunique()
    
    all_cols = [c for c in df.columns if c not in ['customer_ID','S_2']]
    cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
    if ohe_cols is None: ohe_cols = list()
    num_features = [col for col in all_cols if col not in cat_features+ohe_cols]

    all_results = list()
    
    print("[INFO] Computing 'after pay' features")
    tic = time.time()
    for bcol in [f'B_{i}' for i in [11,14,17]]+['D_39','D_131']+[f'S_{i}' for i in [16,23]]:
        for pcol in ['P_2','P_3']:
            if bcol in df.columns:
                df[f'{bcol}-{pcol}'] = df[bcol] - df[pcol]
                num_features.append(f'{bcol}-{pcol}')
    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")

    
    print("[INFO] Computing numerical aggregations")
    tic = time.time()
    df_num_agg = df.groupby("customer_ID")[num_features].agg(['first', 'mean', 'median', 'std', 'min', 'max', 'last'])
    df_num_agg.columns = ['_'.join(x) for x in df_num_agg.columns]
    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")

    
    print("[INFO] Computing diff_last-* features")
    tic = time.time()
    for col in num_features:
        df_num_agg[f"{col}_diff_last-first"] = df_num_agg[f"{col}_last"] - df_num_agg[f"{col}_first"]
        df_num_agg[f"{col}_diff_last-mean"] = df_num_agg[f"{col}_last"] - df_num_agg[f"{col}_mean"]  
        df_num_agg[f"{col}_diff_last-median"] = df_num_agg[f"{col}_last"] - df_num_agg[f"{col}_median"]
        df_num_agg[f"{col}_diff_last-min"] = df_num_agg[f"{col}_last"] - df_num_agg[f"{col}_min"]
        df_num_agg[f"{col}_diff_last-max"] = df_num_agg[f"{col}_last"] - df_num_agg[f"{col}_max"]

    to_remove = list(filter(re.compile(".*_first").match, df_num_agg.columns))
    df_num_agg.drop(to_remove, axis=1, inplace=True)

    df_num_agg = reduce_mem_usage(df_num_agg, verbose=True)
    gc.collect()

    assert n_customers == df_num_agg.shape[0]
    all_results.append(df_num_agg)

    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")
    
    
    print("[INFO] Computing above/below mean")
    tic = time.time()
    
    df_tmp = df.groupby("customer_ID")[num_features].transform("median")

    df_above_mean = df[num_features] > df_tmp
    df_above_mean["customer_ID"] = df["customer_ID"]

    df_below_mean = df[num_features] < df_tmp
    df_below_mean["customer_ID"] = df["customer_ID"]

    df_above_mean_agg = df_above_mean.groupby("customer_ID")[num_features].sum()
    df_above_mean_agg.columns = [col+"_above_mean" for col in df_above_mean_agg.columns]

    df_below_mean_agg = df_below_mean.groupby("customer_ID")[num_features].sum()
    df_below_mean_agg.columns = [col+"_below_mean" for col in df_below_mean_agg.columns]
    
    with Parallel(n_jobs=-1) as parallel:
        delayed_func = delayed(count_consecutives)
        results = parallel(
           delayed_func(_df, customer_ID, num_features) 
           for customer_ID,_df in tqdm(df_above_mean.groupby("customer_ID"))
        )
    df_above_mean_cc = pd.DataFrame(results).set_index("customer_ID")
    df_above_mean_cc.columns = [f"{col}_above_mean_cc" for col in df_above_mean_cc.columns]

    with Parallel(n_jobs=-1) as parallel:
        delayed_func = delayed(count_consecutives)
        results = parallel(
           delayed_func(_df, customer_ID, num_features) 
           for customer_ID,_df in tqdm(df_below_mean.groupby("customer_ID"))
        )
    df_below_mean_cc = pd.DataFrame(results).set_index("customer_ID")
    df_below_mean_cc.columns = [f"{col}_below_mean_cc" for col in df_below_mean_cc.columns]
    
    df_above_mean_agg = reduce_mem_usage(df_above_mean_agg, verbose=True)
    df_below_mean_agg = reduce_mem_usage(df_below_mean_agg, verbose=True)
    df_above_mean_cc = reduce_mem_usage(df_above_mean_cc, verbose=True)
    df_below_mean_cc = reduce_mem_usage(df_below_mean_cc, verbose=True)
    gc.collect()

    assert n_customers == df_above_mean_agg.shape[0]
    all_results.append(df_above_mean_agg)
    assert n_customers == df_below_mean_agg.shape[0]
    all_results.append(df_below_mean_agg)
    assert n_customers == df_above_mean_cc.shape[0]
    all_results.append(df_above_mean_cc)
    assert n_customers == df_below_mean_cc.shape[0]
    all_results.append(df_below_mean_cc)

    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")


    print("[INFO] Computing diff features")
    tic = time.time()
    df_diff = df.groupby("customer_ID")[num_features].diff()
    df_diff["customer_ID"] = df["customer_ID"]
    df_pchg = df.replace({0.:1e-10}).groupby("customer_ID")[num_features].pct_change()
    df_pchg["customer_ID"] = df["customer_ID"]

    df_diff_agg = df_diff.groupby("customer_ID")[num_features].agg(["mean","std","min","max",])
    df_diff_agg.columns = ['_diff_'.join(x) for x in df_diff_agg.columns]

    df_pchg_agg = df_pchg.groupby("customer_ID")[num_features].agg(["mean","std","min","max"])
    df_pchg_agg.columns = ['_pchg_'.join(x) for x in df_pchg_agg.columns]

    df_diff_agg = reduce_mem_usage(df_diff_agg, verbose=True)
    df_pchg_agg = reduce_mem_usage(df_pchg_agg, verbose=True)
    gc.collect()

    assert n_customers == df_diff_agg.shape[0]
    all_results.append(df_diff_agg)
    assert n_customers == df_pchg_agg.shape[0]
    all_results.append(df_pchg_agg)

    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")
    
    
    print("[INFO] Computing diff/pchg lag features")
    tic = time.time()
    # lags in diff features
    df_diff_lag1 = df_diff.groupby("customer_ID")[num_features].nth(-1, dropna=None)
    df_diff_lag1.columns = [f"{col}_diff_lag1" for col in df_diff_lag1.columns]

    df_diff_lag2 = df_diff.groupby("customer_ID")[num_features].nth(-2, dropna=None)
    df_diff_lag2.columns = [f"{col}_diff_lag2" for col in df_diff_lag2.columns]

    df_diff_lag3 = df_diff.groupby("customer_ID")[num_features].nth(-3, dropna=None)
    df_diff_lag3.columns = [f"{col}_diff_lag3" for col in df_diff_lag3.columns]

    df_diff_lag = pd.concat([df_diff_lag1, df_diff_lag2, df_diff_lag3], axis=1)

    # lags on pct_change features
    df_pchg_lag1 = df_pchg.groupby("customer_ID")[num_features].nth(-1, dropna=None)
    df_pchg_lag1.columns = [f"{col}_pchg_lag1" for col in df_pchg_lag1.columns]

    df_pchg_lag2 = df_pchg.groupby("customer_ID")[num_features].nth(-2, dropna=None)
    df_pchg_lag2.columns = [f"{col}_pchg_lag2" for col in df_pchg_lag2.columns]

    df_pchg_lag3 = df_pchg.groupby("customer_ID")[num_features].nth(-3, dropna=None)
    df_pchg_lag3.columns = [f"{col}_pchg_lag3" for col in df_pchg_lag3.columns]

    df_pchg_lag = pd.concat([df_pchg_lag1, df_pchg_lag2, df_pchg_lag3], axis=1)

    df_diff_lag = reduce_mem_usage(df_diff_lag, verbose=True)
    df_pchg_lag = reduce_mem_usage(df_pchg_lag, verbose=True)
    gc.collect()

    assert n_customers == df_diff_lag.shape[0]
    all_results.append(df_diff_lag)
    assert n_customers == df_pchg_lag.shape[0]
    all_results.append(df_pchg_lag)

    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")
    
    
    print("[INFO] computing complexity features")
    tic = time.time()
    
    df_diff_sq = df_diff[num_features]**2
    df_diff_sq["customer_ID"] = df_diff["customer_ID"]

    df_pchg_sq = df_pchg[num_features]**2
    df_pchg_sq["customer_ID"] = df_pchg["customer_ID"]

    df_cxty1 = df_diff_sq.groupby("customer_ID")[num_features].agg(["sum","mean"])
    df_cxty1.columns = ["_".join(col)+"_cxty1" for col in df_cxty1.columns]
    
    df_cxty2 = df_pchg_sq.groupby("customer_ID")[num_features].agg(["sum","mean"])
    df_cxty2.columns = ["_".join(col)+"_cxty2" for col in df_cxty2.columns]
    
    df_cxty1 = reduce_mem_usage(df_cxty1, verbose=True)
    df_cxty2 = reduce_mem_usage(df_cxty2, verbose=True)
    gc.collect()
    
    assert n_customers == df_cxty1.shape[0]
    all_results.append(df_cxty1)
    assert n_customers == df_cxty2.shape[0]
    all_results.append(df_cxty2)

    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")


    print("[INFO] computing min_max_position features")
    tic = time.time()

    with Parallel(n_jobs=-1) as parallel:
        delayed_func = delayed(min_max_position)
        results = parallel(
            delayed_func(_df, customer_ID, num_features) 
            for customer_ID,_df in tqdm(df.groupby("customer_ID"))
        )
    df_min_max1 = pd.DataFrame(results).set_index("customer_ID")

    df_diff.rename(columns={col:col+"_diff" for col in num_features}, inplace=True)
    num_features_diff = [col+"_diff" for col in num_features]
    with Parallel(n_jobs=-1) as parallel:
        delayed_func = delayed(min_max_position)
        results = parallel(
            delayed_func(_df, customer_ID, num_features_diff) 
            for customer_ID,_df in tqdm(df_diff.groupby("customer_ID"))
        )
    df_min_max2 = pd.DataFrame(results).set_index("customer_ID")

    df_min_max1 = reduce_mem_usage(df_min_max1, verbose=True)
    df_min_max2 = reduce_mem_usage(df_min_max2, verbose=True)
    gc.collect()

    assert n_customers == df_min_max1.shape[0]
    all_results.append(df_min_max1)
    assert n_customers == df_min_max2.shape[0]
    all_results.append(df_min_max2)

    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")

    
    print("[INFO] Computing categorical aggregations")
    tic = time.time()

    df_cat_agg1 = (
        df
        .groupby("customer_ID")
        [ohe_cols]
        .mean()
    )
    df_cat_agg1.columns = [col+"_"+"mean" for col in df_cat_agg1.columns]

    df_cat_agg2 = (
        df
        .groupby("customer_ID")
        [ohe_cols]
        .agg(compute_last_observed)
    )
    df_cat_agg2.columns = [col+"_"+"lo" for col in df_cat_agg2.columns]

    df_cat_agg1 = reduce_mem_usage(df_cat_agg1, verbose=True)
    df_cat_agg2 = reduce_mem_usage(df_cat_agg2, verbose=True)
    gc.collect()

    assert n_customers == df_cat_agg1.shape[0]
    all_results.append(df_cat_agg1)
    assert n_customers == df_cat_agg2.shape[0]
    all_results.append(df_cat_agg2)
    
    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")

    
    print("[INFO] Computing slope features")
    tic = time.time()
    with Parallel(n_jobs=-1) as parallel:
        delayed_func = delayed(compute_slope_cols)
        results = parallel(
           delayed_func(_df, customer_ID, num_features) 
           for customer_ID,_df in tqdm(df.groupby("customer_ID"))
        )
    slopes_df = pd.DataFrame(results).fillna(0).set_index("customer_ID")
    slopes_df.columns = [f"{col}_slope" for col in slopes_df.columns]

    slopes_df = reduce_mem_usage(slopes_df, verbose=True)
    gc.collect()

    assert n_customers == slopes_df.shape[0]
    all_results.append(slopes_df)

    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")
    
    
    print("[INFO] Building NaN related features")
    tic = time.time()

    df_nan = df[num_features].isna()
    df_nan["customer_ID"] = df["customer_ID"]

    df_nan_agg = df_nan.groupby("customer_ID")[num_features].agg(["mean", "sum"])
    df_nan_agg.columns = ["_".join(col)+"_"+"nan" for col in df_nan_agg.columns]

    df_nan_fpo = df_nan.groupby("customer_ID")[num_features].agg(first_nan_position)
    df_nan_fpo.columns = [col+"_"+"nan_fpo" for col in df_nan_fpo.columns]

    df_nan_lpo = df_nan.groupby("customer_ID")[num_features].agg(last_nan_position)
    df_nan_lpo.columns = [col+"_"+"nan_lpo" for col in df_nan_lpo.columns]

    df_nan_agg = reduce_mem_usage(df_nan_agg, verbose=True)
    df_nan_fpo = reduce_mem_usage(df_nan_fpo, verbose=True)
    df_nan_lpo = reduce_mem_usage(df_nan_lpo, verbose=True)
    gc.collect()

    assert n_customers == df_nan_agg.shape[0]
    all_results.append(df_nan_agg)
    assert n_customers == df_nan_fpo.shape[0]
    all_results.append(df_nan_fpo)
    assert n_customers == df_nan_lpo.shape[0]
    all_results.append(df_nan_lpo)
    
    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")

    
    print("[INFO] Building S_2 related features")
    tic = time.time()
    
    df_count = df.groupby(["customer_ID"])["S_2"].count()
    df_count = pd.DataFrame(df_count).rename({"S_2":"S_2_count"}, axis=1)
    
    _tmp = df.groupby("customer_ID").apply(compute_antiquity)
    df_antiquity = pd.DataFrame(_tmp, columns=["S_2_antiquity"])
    
    _tmp = df.groupby("customer_ID")["S_2"].apply(lambda x: x.diff().max().days)
    df_max_gap = pd.DataFrame(_tmp).rename({"S_2":"S_2_max_gap"}, axis=1)

    df_count = reduce_mem_usage(df_count, verbose=True)
    df_antiquity = reduce_mem_usage(df_antiquity, verbose=True)
    df_max_gap = reduce_mem_usage(df_max_gap, verbose=True)
    gc.collect()

    assert n_customers == df_count.shape[0]
    all_results.append(df_count)
    assert n_customers == df_antiquity.shape[0]
    all_results.append(df_antiquity)
    assert n_customers == df_max_gap.shape[0]
    all_results.append(df_max_gap)

    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")


    print("[INFO] Concatenating all the results")
    tic = time.time()
    output = pd.concat(all_results, axis=1)
    tac = time.time()
    print(f"Elapsed time: {(tac-tic)/60} min")
    
    del df
    del df_num_agg, df_diff, df_pchg, df_diff_agg, df_pchg_agg, slopes_df
    del df_min_max1, df_min_max2, df_cat_agg1, df_cat_agg2, df_count
    gc.collect()

    print('[INFO] shape after engineering', output.shape )
    
    return output

***
## preproc on train

In [ ]:
train = pd.read_parquet("../data/ext/amex-data-integer-dtypes-parquet-format/train.parquet")
train.info()

In [ ]:
print(list(filter(re.compile("D_.*").match, train.columns)))
print("len:", len(list(filter(re.compile("D_.*").match, train.columns))))

In [ ]:
print(list(filter(re.compile("S_.*").match, train.columns)))
print("len:", len(list(filter(re.compile("S_.*").match, train.columns))))

In [ ]:
print(list(filter(re.compile("P_.*").match, train.columns)))
print("len:", len(list(filter(re.compile("P_.*").match, train.columns))))

In [ ]:
print(list(filter(re.compile("B_.*").match, train.columns)))
print("len:", len(list(filter(re.compile("B_.*").match, train.columns))))

In [ ]:
print(list(filter(re.compile("R_.*").match, train.columns)))
print("len:", len(list(filter(re.compile("R_.*").match, train.columns))))

In [ ]:
%%time
train = remove_noise(train)
train, encoder, ohe_cols = encode_categoricals(train)
train_agg = build_features(train, ohe_cols)
train_agg = reduce_mem_usage(train_agg, verbose=True)
train_agg.to_parquet(str(OUTPUT_PATH/"train.parquet"))

del train,train_agg
gc.collect()

***
## preproc on test

In [ ]:
#test = pd.read_parquet("../data/ext/amex-data-integer-dtypes-parquet-format/test.parquet")
#test.info()

In [ ]:
#%%time
#test = remove_noise(test)
#test, _, _ = encode_categoricals(test, encoder)
#test_agg = build_features(test, ohe_cols)
#test_agg = reduce_mem_usage(test_agg, verbose=True)
#test_agg.to_parquet(str(OUTPUT_PATH/"test.parquet"))

#del test,test_agg
#gc.collect()

***